In [1]:
import pandas as pd
import numpy as np
import datasets

In [2]:
from datasets import load_dataset

# ETL of Large Source Dataset
Before uploading the dataset into a database and dataloading pipeline, the data has to be converted into I.I.E (independent and identifiable data) form by including a unique I.D and timestamp for each row. The data will also be split into its' train and validation subsets.

In [3]:
tinystories = "skeskinen/TinyStories-GPT4"
train_data = load_dataset(tinystories, split="train[:2200000]").shuffle()
val_data = load_dataset(tinystories, split="train[2200000:]").shuffle()

(…)-00000-of-00008-c63ccd5d5290f4a1.parquet:  54%|#####3    | 105M/194M [00:00<?, ?B/s]

(…)-00001-of-00008-478199d8ac044910.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00002-of-00008-9b868f59be94d815.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00003-of-00008-d183cca02834cd90.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00004-of-00008-5f8ac0bb66de5834.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00005-of-00008-e8c22c3e776b87dd.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00006-of-00008-941f57106aca3340.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

(…)-00007-of-00008-771d8aa2d5ce5c95.parquet:   0%|          | 0.00/194M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/2745100 [00:00<?, ? examples/s]

In [4]:
train_data

Dataset({
    features: ['story', 'summary', 'source', 'prompt', 'words', 'features'],
    num_rows: 2200000
})

In [5]:
val_data

Dataset({
    features: ['story', 'summary', 'source', 'prompt', 'words', 'features'],
    num_rows: 545100
})

In [8]:
# vectorized pandas df
train_df = pd.DataFrame(train_data)
val_df = pd.DataFrame(val_data)

# columns added to train and val
train_df['unique_id'] = range(len(train_df))  # Unique IDs
train_df['timestamp'] = pd.to_datetime('now')  # Current timestamp

val_df['unique_id'] = range(len(val_df))  # Unique IDs
val_df['timestamp'] = pd.to_datetime('now')  # Current timestamp

KeyboardInterrupt: 